# Company webpage

Extract some key information from the company webpage

In [1]:
from core import CompanyProduct, init

init()

In [2]:
target = CompanyProduct.same("98point6")

In [3]:
from core import make_experiment_dir

experiment_dir = make_experiment_dir(target)

In [4]:
from google_search import search

results = list(search("site:98point6.com", num=20))
results

2024-08-27 16:56:31.070 | DEBUG    | google_search:search:58 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - site:98point6.com', 'totalResults': '46', 'searchTerms': 'site:98point6.com', 'count': 10, 'startIndex': 1, 'langua

[SearchResult(title='98point6 Virtual Care Platform for async and real-time telehealth', link='https://www.98point6.com/', snippet='98point6 empowers health systems to decrease the administrative burden on clinicians, promote quality, and increase patient satisfaction.', formattedUrl='https://www.98point6.com/'),
 SearchResult(title='Newsroom | 98point6 Technologies', link='https://www.98point6.com/newsroom/', snippet="As it embarks on a $32 million-fueled business pivot, care enabler 98point6 bought telehealth company Bright.md's customers, CEO Jay Burrell tells Axios\xa0...", formattedUrl='https://www.98point6.com/newsroom/'),
 SearchResult(title='Platform | 98point6 Technologies', link='https://www.98point6.com/platform/', snippet='The 98point6 virtual care platform. Treat patients in real-time and asynchronously to ensure they get the care they need, when they need it — while streamlining\xa0...', formattedUrl='https://www.98point6.com/platform/'),
 SearchResult(title='Blog | 98poi

In [5]:
# request and get the content
from news.scrape import request_article, response_to_article

responses = [request_article(result.link) for result in results]
contents = [response_to_article(response) for response in responses]

In [6]:
from news.scrape import article_to_markdown

for article in contents:
    print(article_to_markdown(article))

# [98point6 Virtual Care Platform for async and real on 2024-02-06](https://www.98point6.com/)
Frictionless virtual care

Imagine automating information gathered from the time the patient walks in the door to the moment they meet the clinician.

If you think thatâs efficient, imagine how streamlined patient care is with decision-support tools.

Now picture that all happening alongside a real-time conversation with the patient via text-based messaging, audio or video. Oh, and did we mention that itâs turn-key and white labeled?

Thatâs the 98point6 difference.
# [98point6 Technologies on 2022-12-07](https://www.98point6.com/newsroom/)
News and Updates

With more than six years of experience, after treating our first patient, 98point6 Technologies is licensing our cloud-based platform so more patients can benefit from increased access to high-quality care. Learn more about us via news coverage of our digital-first care delivery model.

98point6 is ditching its telehealth business t

In [7]:
joined_markdown = "\n\n".join([article_to_markdown(article) for article in contents if len(article.text) > 500])

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages.ai import AIMessage

from pydantic import BaseModel, Field
from typing import List

_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You'll be given several pages from a company website in markdown format.
Read all the pages carefully and summarize all the content in markdown format.
Include citations for all statements in the summary.
            """,
        ),
        (
            "human",
            """
Webpages:
{context}

Summary in markdown format:
            """,
        ),
    ]
)



llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
runnable = _prompt | llm
result = runnable.invoke(
    {
        "context": joined_markdown,
    }
)

result

AIMessage(content='# Summary of 98point6 Technologies\n\n## Overview\n98point6 Technologies is a healthcare technology company that specializes in providing virtual care solutions through an AI-powered, text-based platform. Founded in 2015 and headquartered in Seattle, the company aims to enhance access to primary care by leveraging technology to deliver efficient and affordable healthcare services.\n\n## Services Offered\n- **Asynchronous Care**: Patients can seek care for non-urgent concerns without scheduling, allowing flexibility in accessing healthcare. The platform supports clinicians in diagnosing and treating over 140 common conditions across all age groups (98point6 Technologies, 2024).\n- **Real-Time Virtual Care**: Offers text, audio, and video interactions, available on-demand or through scheduled visits. Patients can report symptoms, and the platform gathers relevant history before connecting them to a clinician (98point6 Technologies, 2024).\n- **AI and Automation**: The 

In [9]:
print(result.content)

# Summary of 98point6 Technologies

## Overview
98point6 Technologies is a healthcare technology company that specializes in providing virtual care solutions through an AI-powered, text-based platform. Founded in 2015 and headquartered in Seattle, the company aims to enhance access to primary care by leveraging technology to deliver efficient and affordable healthcare services.

## Services Offered
- **Asynchronous Care**: Patients can seek care for non-urgent concerns without scheduling, allowing flexibility in accessing healthcare. The platform supports clinicians in diagnosing and treating over 140 common conditions across all age groups (98point6 Technologies, 2024).
- **Real-Time Virtual Care**: Offers text, audio, and video interactions, available on-demand or through scheduled visits. Patients can report symptoms, and the platform gathers relevant history before connecting them to a clinician (98point6 Technologies, 2024).
- **AI and Automation**: The platform uses AI to automat

In [10]:
import os

# Define the file path
file_path = os.path.join(experiment_dir, "summary.md")

# Write the content to the markdown file
with open(file_path, "w") as file:
    file.write(result.content)

print("Content written to markdown file:", file_path)

Content written to markdown file: /home/keith/company-detective/output/98point6_98point6/2024-08-27/summary.md
